In [4]:
# load module and pre generate data
from graph_tree_list_module import *

import numpy as np

import pickle

# Open the file in binary read mode and load the object
with open("graph_tree.pkl", "rb") as file:
    gt = pickle.load(file)


In [5]:
sliders_data = [{"id": gt.attr_name[i], "min": gt.X_maxmin[1][i], "max": gt.X_maxmin[0][i], "value": gt.nodes[0].center[i], "type": "f"} for i in range(len(gt.attr_name))]
# print(sliders_data)

recom_data = [{"i": i, "id": gt.attr_name[i], "max_val": gt.recom[i][0].center[i], "min_val": gt.recom[i][1].center[i]} for i in range(len(gt.attr_name))]
# print(recom_data)

graph_tree_horizontal_data, graph_tree_vertical_data = gt.get_graph_tree_svg_data(node_id = 127)

tree_data = {
        "name": "Root",
        "children": [
            {
                "name": "Child 1",
                "children": [
                    {"name": "Grandchild 1"},
                    {"name": "Grandchild 2"}
                ]
            },
            {
                "name": "Child 2",
                "children": [
                    {"name": "Grandchild 3"},
                    {"name": "Grandchild 4"}
                ]
            }
        ]
    }

reflection_node: 90 at lv up: 2
generate cos sim, ecul dist, and nodes sorted by cos sim
[{'id': 119, 'cos_sim': np.float64(-0.4954057176696461), 'euclide_dist': np.float64(0.45544397974559686), 'r_ratio': 0.6111111111111112, 'cluster_type': 'neighbour'}, {'id': 126, 'cos_sim': np.float64(-0.40089936891980577), 'euclide_dist': np.float64(0.30260152142167707), 'r_ratio': 1.0833333333333333, 'cluster_type': 'neighbour'}, {'id': 127, 'cos_sim': np.float64(1.0), 'euclide_dist': np.float64(0.0), 'r_ratio': 1.0, 'cluster_type': 'center'}]
[{'id': 118, 'r_ratio': 2.0833333333333335, 'dy': 0, 'dx': 0, 'cluster_type': 'parent'}]


In [ ]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

# slider ####################################################################
@app.route('/sliders')
def sliders():
    return jsonify(sliders_data)

@app.route('/handle_slider', methods=['POST'])
def submit_slider_values():
    sucess_msg = "slider respon success"
    data = request.json
    print('Received slider values:', data)
    query = np.array([float(val) for val in data.values()])
    cur = gt.search(query)
    graph_tree_horizontal_data, graph_tree_vertical_data = gt.get_graph_tree_svg_data(cur.id)
    # Process the data as needed
    return jsonify({'status': sucess_msg,
                    "graph_tree_data": {"graph_tree_horizontal_data": graph_tree_horizontal_data, "graph_tree_vertical_data": graph_tree_vertical_data}})
# slider end ################################################################

# recom ####################################################################
@app.route('/recom')
def recom():
    return jsonify(recom_data)

@app.route('/handle_recom', methods=['POST'])
def handle_recom():
    sucess_msg = "recom respon success"
    data = request.json
    button_id = int(data.get('buttonId'))
    val = int(data.get('val'))
    # print(button_id, val)
    moved_node = gt.recom[button_id][val]
    graph_tree_horizontal_data, graph_tree_vertical_data = gt.get_graph_tree_svg_data(moved_node.id)
    # print(graph_tree_data)
    return jsonify({'status': sucess_msg,
                    'buttonId': button_id,
                    'val': val,
                    "graph_tree_data": {"graph_tree_horizontal_data": graph_tree_horizontal_data, "graph_tree_vertical_data": graph_tree_vertical_data}})
# recom end ####################################################################

@app.route('/graph_tree_data')
def graph():
    return jsonify({"graph_tree_data": {"graph_tree_horizontal_data": graph_tree_horizontal_data, "graph_tree_vertical_data": graph_tree_vertical_data}})

@app.route('/graph_tree_circle_click', methods=['POST'])
def graph_tree_circle_click():
    sucess_msg = "graph_tree_circle_click respon success"
    data = request.json
    id = int(data.get('id'))
    print(id)
    graph_tree_horizontal_data, graph_tree_vertical_data = gt.get_graph_tree_svg_data(id)
    return jsonify({'status': sucess_msg,
                    'id': id,
                    "graph_tree_data": {"graph_tree_horizontal_data": graph_tree_horizontal_data, "graph_tree_vertical_data": graph_tree_vertical_data}})


@app.route('/tree_data')
def tree():
    return jsonify(tree_data)

if __name__ == '__main__':
    app.run(port=8123)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8123
Press CTRL+C to quit
127.0.0.1 - - [16/Oct/2024 00:48:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 00:48:54] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 00:48:54] "GET /static/graph_tree_shared_variable.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 00:48:54] "GET /static/curve_axis.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 00:48:54] "GET /static/scripts.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 00:48:54] "GET /static/graph_tree.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 00:48:54] "GET /sliders HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 00:48:54] "GET /recom HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 00:48:54] "GET /graph_tree_data HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 00:48:54] "GET /tree_data HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 00:48:57] "POST /graph_tree_circle_click HTTP/1.1" 200 -


126
reflection_node: 90 at lv up: 2
generate cos sim, ecul dist, and nodes sorted by cos sim
[{'id': 119, 'cos_sim': np.float64(-0.597192351957355), 'euclide_dist': np.float64(0.4688534466597052), 'r_ratio': 0.5641025641025641, 'cluster_type': 'neighbour'}, {'id': 127, 'cos_sim': np.float64(-0.40089936891980577), 'euclide_dist': np.float64(0.30260152142167707), 'r_ratio': 0.9230769230769231, 'cluster_type': 'neighbour'}, {'id': 126, 'cos_sim': np.float64(1.0), 'euclide_dist': np.float64(0.0), 'r_ratio': 1.0, 'cluster_type': 'center'}]
[{'id': 118, 'r_ratio': 1.9230769230769231, 'dy': 0, 'dx': 0, 'cluster_type': 'parent'}]


127.0.0.1 - - [16/Oct/2024 00:53:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 00:53:17] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 00:53:17] "GET /static/graph_tree_shared_variable.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 00:53:17] "GET /static/curve_axis.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 00:53:17] "GET /static/scripts.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 00:53:17] "GET /static/graph_tree.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 00:53:17] "GET /sliders HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 00:53:17] "GET /recom HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 00:53:17] "GET /graph_tree_data HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 00:53:17] "GET /tree_data HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 00:53:19] "POST /graph_tree_circle_click HTTP/1.1" 200 -


126
reflection_node: 90 at lv up: 2
generate cos sim, ecul dist, and nodes sorted by cos sim
[{'id': 119, 'cos_sim': np.float64(-0.597192351957355), 'euclide_dist': np.float64(0.4688534466597052), 'r_ratio': 0.5641025641025641, 'cluster_type': 'neighbour'}, {'id': 127, 'cos_sim': np.float64(-0.40089936891980577), 'euclide_dist': np.float64(0.30260152142167707), 'r_ratio': 0.9230769230769231, 'cluster_type': 'neighbour'}, {'id': 126, 'cos_sim': np.float64(1.0), 'euclide_dist': np.float64(0.0), 'r_ratio': 1.0, 'cluster_type': 'center'}]
[{'id': 118, 'r_ratio': 1.9230769230769231, 'dy': 0, 'dx': 0, 'cluster_type': 'parent'}]


...

127.0.0.1 - - [16/Oct/2024 02:01:35] "POST /graph_tree_circle_click HTTP/1.1" 200 -


126
reflection_node: 90 at lv up: 2
generate cos sim, ecul dist, and nodes sorted by cos sim
[{'id': 119, 'cos_sim': np.float64(-0.597192351957355), 'euclide_dist': np.float64(0.4688534466597052), 'r_ratio': 0.5641025641025641, 'cluster_type': 'neighbour'}, {'id': 127, 'cos_sim': np.float64(-0.40089936891980577), 'euclide_dist': np.float64(0.30260152142167707), 'r_ratio': 0.9230769230769231, 'cluster_type': 'neighbour'}, {'id': 126, 'cos_sim': np.float64(1.0), 'euclide_dist': np.float64(0.0), 'r_ratio': 1.0, 'cluster_type': 'center'}]
[{'id': 118, 'r_ratio': 1.9230769230769231, 'dy': 0, 'dx': 0, 'cluster_type': 'parent'}]


127.0.0.1 - - [16/Oct/2024 02:01:40] "POST /graph_tree_circle_click HTTP/1.1" 200 -


119
reflection_node: 40 at lv up: 2
generate cos sim, ecul dist, and nodes sorted by cos sim
[{'id': 117, 'cos_sim': np.float64(-0.5183294381304352), 'euclide_dist': np.float64(0.5735584537360862), 'r_ratio': 1.4090909090909092, 'cluster_type': 'neighbour'}, {'id': 116, 'cos_sim': np.float64(-0.20707014964952927), 'euclide_dist': np.float64(0.5270678452813375), 'r_ratio': 1.7727272727272727, 'cluster_type': 'neighbour'}, {'id': 118, 'cos_sim': np.float64(-0.08732282252906076), 'euclide_dist': np.float64(0.43705718179820374), 'r_ratio': 3.409090909090909, 'cluster_type': 'neighbour'}, {'id': 119, 'cos_sim': np.float64(1.0), 'euclide_dist': np.float64(0.0), 'r_ratio': 1.0, 'cluster_type': 'center'}]
[{'id': 90, 'r_ratio': 4.409090909090909, 'dy': 0, 'dx': 0, 'cluster_type': 'parent'}]


127.0.0.1 - - [16/Oct/2024 02:01:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 02:01:53] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 02:01:53] "GET /static/graph_tree_shared_variable.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 02:01:53] "GET /static/curve_axis.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 02:01:53] "GET /static/graph_tree.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 02:01:53] "GET /static/scripts.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 02:01:53] "GET /sliders HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 02:01:53] "GET /recom HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 02:01:53] "GET /graph_tree_data HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 02:01:53] "GET /tree_data HTTP/1.1" 200 -
